In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.options import Options


In [1]:

options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Uncomment to run headlessly after confirming the script works visually
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://www.fotmob.com/leagues/47/stats/season/20720/players/goals/premier-league-players"
driver.get(url)

all_links = []

try:
    wait = WebDriverWait(driver, 10) 
    while True:
        
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.css-fvfi51-LeagueSeasonStatsTableCSS.eybz1nb15 a')))

      
        links = driver.find_elements(By.CSS_SELECTOR, '.css-fvfi51-LeagueSeasonStatsTableCSS.eybz1nb15 a')
        for link in links:
            all_links.append(link.get_attribute('href'))

      
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'css-fvfi51-LeagueSeasonStatsTableCSS')]//button[contains(., 'Next')]")))
        if next_button:
            next_button.click()
            
            time.sleep(5)  
        else:
            break
except Exception as e:
    print(f"Error navigating pages: {str(e)}")
finally:
    driver.quit()



Error navigating pages: Message: 



In [5]:

option = Options()
option.headless = True

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service= service, options= options)

url = "https://www.fotmob.com/players/737066/erling-haaland"

driver.get(url)

driver.implicitly_wait(10)

html = driver.page_source
driver.quit()

soup = BeautifulSoup(html, 'html.parser')

def extract_stat(soup, state_title):
    state_item  = soup.find('div',class_='css-2duihq-StatTitle', text=state_title)
    if state_item:
        return state_item.find_next('div', class_ = 'css-6xxhwd-StatValue').span.text.strip()
    return "Not Available"
stats_titles = ['Goals', 'Assists', 'Expected goals (xG)', 'Shots', 'Successful passes', 'Shots on target', 'Penalty goals', 'Expected assists (xA)',
    'Pass accuracy', 'Accurate long balls', 'Long ball accuracy', 'Chances created', 'Successful dribbles', 'Dribble success', 'Touches',
    'Touches in opposition box', 'Dispossessed', 'Fouls won', 'Penalties awarded', 'Tackles won', 'Tackles won %', 'Duels won', 'Duels won %',
    'Aerial duels won', 'Aerial duels won %', 'Interceptions', 'Blocked', 'Fouls committed', 'Recoveries', 'Possession won final 3rd', 'Dribbled past',
    'Yellow cards', 'Red cards']      

state_data = {title: extract_stat(soup, title) for title in stats_titles}  
        
        
    


C:\Users\idree\AppData\Local\Temp\ipykernel_18336\4058507847.py:28: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  state_item  = soup.find('div',class_='css-2duihq-StatTitle', text=state_title)


In [12]:

options = Options()
options.headless = True
service = Service(ChromeDriverManager().install())

# List of player URLs
player_urls = all_links

# Stats titles
stats_titles = ['Goals', 'Assists', 'Expected goals (xG)', 'Shots', 'Successful passes', 'Shots on target', 'Penalty goals', 'Expected assists (xA)',
    'Pass accuracy', 'Accurate long balls', 'Long ball accuracy', 'Chances created', 'Successful dribbles', 'Dribble success', 'Touches',
    'Touches in opposition box', 'Dispossessed', 'Fouls won', 'Penalties awarded', 'Tackles won', 'Tackles won %', 'Duels won', 'Duels won %',
    'Aerial duels won', 'Aerial duels won %', 'Interceptions', 'Blocked', 'Fouls committed', 'Recoveries', 'Possession won final 3rd', 'Dribbled past',
    'Yellow cards', 'Red cards'
                ]

all_players_stats = []


for url in player_urls:
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    driver.implicitly_wait(10) 
    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')


    name_tag = soup.find('h1', class_='css-xgh3st-PlayerNameCSS')
    if name_tag:
        player_name = name_tag.text.strip()
    else:
        print(f"Failed to find the player's name for URL: {url}")
        player_name = "Unknown"


    player_stats = {'Name': player_name}


    player_stats.update({title: extract_stat(soup, title) for title in stats_titles})

    all_players_stats.append(player_stats)




C:\Users\idree\AppData\Local\Temp\ipykernel_18336\1850303908.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  state_item  = soup.find('div',class_='css-2duihq-StatTitle', text=state_title)


In [19]:
df = pd.DataFrame(all_players_stats)

df

,Name,Goals,Assists,Expected goals (xG),Shots,Successful passes,Shots on target,Penalty goals,Expected assists (xA),Pass accuracy,...,Aerial duels won,Aerial duels won %,Interceptions,Blocked,Fouls committed,Recoveries,Possession won final 3rd,Dribbled past,Yellow cards,Red cards
0,Erling Haaland,21,5,24.12,104,252,50,4,2.09,77.3%,...,34,47.2%,2,19,15,39,19,1,1,0
1,Cole Palmer,20,9,15.56,88,930,41,9,6.71,82.9%,...,1,20.0%,17,24,22,88,25,11,7,0
2,Ollie Watkins,19,12,16.04,103,416,47,Not Available,4.08,76.5%,...,45,36.9%,4,20,20,63,19,6,4,0
3,Alexander Isak,19,1,17.27,66,321,36,5,2.05,79.3%,...,7,17.5%,5,13,13,33,8,6,1,0
4,Dominic Solanke,18,3,18.58,102,374,36,2,1.98,74.4%,...,89,41.0%,7,30,40,93,29,16,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,Wataru Endo,1,0,0.49,9,994,2,Not Available,1.43,87.7%,...,31,53.4%,19,4,42,88,6,32,9,0
284,Willy Boly,1,0,1.42,13,405,5,Not Available,0.68,77.7%,...,49,66.2%,22,2,19,71,2,10,1,1
285,Youri Tielemans,1,6,1.14,13,753,5,Not Available,2.08,86.0%,...,13,59.1%,14,5,22,94,17,22,3,0
286,Fabio Vieira,1,2,1.25,7,119,2,1,0.82,88.8%,...,1,100.0%,2,1,8,16,3,Not Available,0,1
